In [ ]:
"""operators.ipynb"""
# Cell 1 - Create two random state vectors (not normalized)

from __future__ import annotations

import typing

import array_to_latex as a2l
import numpy as np
from IPython.core.display import Math

if typing.TYPE_CHECKING:
    from numpy.typing import NDArray

%matplotlib widget

np.random.seed(2016)
ndims = 5

psi: NDArray[np.complex_] = np.random.random(ndims) + np.random.random(ndims) * 1j
phi: NDArray[np.complex_] = np.random.random(ndims) + np.random.random(ndims) * 1j

display(
    Math(
        (
            r"\mathbf{\lvert\psi\rangle}="
            f"{a2l.to_ltx(psi[:, np.newaxis], frmt='{:6.5f}', print_out=False)}$"
        )
    )
)

display(
    Math(
        (
            r"\mathbf{\lvert\phi\rangle}="
            f"{a2l.to_ltx(phi[:, np.newaxis], frmt='{:6.5f}', print_out=False)}$"
        )
    )
)

In [ ]:
# Cell 2 - Create a Hermitian Operator (as a matrix)


def create_hermitian_matrix(ndims: int) -> NDArray[np.complex_]:
    a: NDArray[np.complex_] = np.zeros((ndims, ndims), dtype=complex)
    for i in range(ndims):
        for j in range(i, ndims):
            r1: float = np.random.random()
            r2: float = np.random.random()
            if i == j:
                a[i, j] = complex(r1, 0)
            else:
                a[i, j] = complex(r1, r2)
                a[j, i] = complex(r1, -r2)
    return a


op: NDArray[np.complex_] = create_hermitian_matrix(ndims)

display(
    Math((r"\mathbf{\hat{O}}=" f"{a2l.to_ltx(op, frmt='{:6.5f}', print_out=False)}$"))
)

In [ ]:
# Cell 3 - A Hermitian operator applied to its eigenkets produce its eigenvalues

eigen_vals: NDArray[np.complex_]
eigen_vecs: NDArray[np.complex_]
eigen_vals, eigen_vecs = np.linalg.eig(op)

# Note: The eigenvalues of a Hermitian operator are all real
display(
    Math(
        (
            r"\mathbf{\lambda}="
            f"{a2l.to_ltx(eigen_vals, frmt='{:6.5f}', print_out=False)}$"
        )
    )
)

# Note: In numpy, eigenvectors are returned as columns
for i in range(ndims):
    display(
        Math(
            (
                rf"\mathbf{{v_{i}}}="
                f"{a2l.to_ltx(eigen_vecs[:,i], frmt='{:6.5f}', print_out=False)}"
            )
        )
    )

bra_phi: NDArray[np.complex_] = phi.conj().T

for i in range(ndims):
    t1: float = np.dot(bra_phi, np.dot(op, eigen_vecs[:, i]))
    t2: float = np.dot(bra_phi, eigen_vals[i] * eigen_vecs[:, i])
    display(
        Math(
            (
                rf"\mathbf{{\langle\phi\lvert\hat{{O}}\lvert v_{i}\rangle="
                rf"\langle\phi\lvert\lambda_{i}\lvert v_{i}\rangle}}\;?\;\rightarrow\;{np.isclose(t1,t2)}"
            )
        )
    )

In [ ]:
# Cell 4 - All non-degenerate eigenvectors
# of a Hermitian operator are orthogonal to each other

for i in range(ndims):
    for j in range(i + 1, ndims):
        display(
            Math(
                (
                    rf"\mathbf{{v_{i}\cdot v_{j}}}\;=\;"
                    rf"{np.dot(eigen_vecs[:,i].conj(),eigen_vecs[:,j]).round(4)}"
                )
            )
        )

In [ ]:
# Cell 5 - Get Matrix From Operator in a given basis


def get_matrix_from_operator(
    op: NDArray[np.complex_], basis: NDArray[np.complex_]
) -> NDArray[np.complex_]:
    m: NDArray[np.complex_] = np.zeros_like(op)

    for i, _ in np.ndenumerate(op):
        row: int = i[0]
        col: int = i[1]
        t1: float = np.dot(basis[row].conj().T, op @ basis[col])
        m[row, col] = t1
    return m


# Create a Hermitian operator matrix
op: NDArray[np.complex_] = np.array([[4, -2], [-2, 4]], dtype=complex)

# Get the eigenvalues and eigenvectors for the operator
eigen_vals: NDArray[np.complex_]
eigen_vecs: NDArray[np.complex_]
eigen_vals, eigen_vecs = np.linalg.eig(op)

# Get the operator's components using its eigenvectors as its basis
m: NDArray[np.complex_] = get_matrix_from_operator(op, eigen_vecs)

display(Math((r"\mathbf{\hat{O}}=" f"{a2l.to_ltx(op, print_out=False)}")))

display(
    Math(
        (
            r"\mathbf{\epsilon}="
            f"{a2l.to_ltx(eigen_vecs, frmt='{:6.5f}', print_out=False)}"
        )
    )
)

display(Math((r"\mathbf{O}=" f"{a2l.to_ltx(m, print_out=False)}")))

display(Math((r"\mathbf{\lambda}=" f"{a2l.to_ltx(eigen_vals, print_out=False)}$")))

In [ ]:
# Cell 6 - Calculate the Commutator

ndims = 3
omega_1: NDArray[np.complex_] = create_hermitian_matrix(ndims)
omega_2: NDArray[np.complex_] = create_hermitian_matrix(ndims)

commutator: NDArray[np.complex_] = np.dot(omega_1, omega_2) - np.dot(omega_2, omega_1)

display(
    Math(
        (r"\mathbf{\Omega_1}=" f"{a2l.to_ltx(omega_1, frmt='{:6.5f}',print_out=False)}")
    )
)

display(
    Math(
        (r"\mathbf{\Omega_2}=" f"{a2l.to_ltx(omega_2, frmt='{:6.5f}',print_out=False)}")
    )
)

display(
    Math(
        (
            r"\mathbf{[\Omega_1,\Omega_2]}="
            f"{a2l.to_ltx(commutator, frmt='{:6.5f}',print_out=False)}"
        )
    )
)


display(
    Math(
        (
            rf"\mathbf{{\Omega_1}}\;\text{{and}}\;\mathbf{{\Omega_2}}\;"
            rf"\text{{commute}}\;?\;\rightarrow\;{np.isclose(commutator,0).all()}"
        )
    )
)

In [ ]:
# Cell 7 - All diagonal matrices commute with each other


def create_diagonal_matrix(ndims: int) -> NDArray[np.complex_]:
    a: NDArray[np.complex_] = np.zeros((ndims, ndims), dtype=complex)
    for i in range(ndims):
        r1: float = np.random.random()
        r2: float = np.random.random()
        a[i, i] = complex(r1, r2)
    return a


omega_1: NDArray[np.complex_] = create_diagonal_matrix(ndims)
omega_2: NDArray[np.complex_] = create_diagonal_matrix(ndims)

commutator: NDArray[np.complex_] = np.dot(omega_1, omega_2) - np.dot(omega_2, omega_1)

display(
    Math(
        (r"\mathbf{\Omega_1}=" f"{a2l.to_ltx(omega_1, frmt='{:6.5f}',print_out=False)}")
    )
)

display(
    Math(
        (r"\mathbf{\Omega_2}=" f"{a2l.to_ltx(omega_2, frmt='{:6.5f}',print_out=False)}")
    )
)

display(
    Math(
        (
            r"\mathbf{[\Omega_1,\Omega_2]}="
            f"{a2l.to_ltx(commutator, frmt='{:6.5f}',print_out=False)}"
        )
    )
)


display(
    Math(
        (
            rf"\mathbf{{\Omega_1}}\;\text{{and}}\;\mathbf{{\Omega_2}}\;"
            rf"\text{{commute}}\;?\;\rightarrow\;{np.isclose(commutator,0).all()}"
        )
    )
)